In [24]:
with driver.session() as session:
    session.run("DROP INDEX chunk_embeddings IF EXISTS")

In [1]:
with driver.session() as session:
    indexes = session.run("SHOW INDEXES").data()
    for idx in indexes:
        # 安全地列印所有欄位，避免 KeyError（不同 Neo4j 版本欄位名稱可能不同）
        try:
            name = idx.get("name")
            typ = idx.get("type") or idx.get("indexType")
            labels = idx.get("labelsOrTypes") or idx.get("entityType")
            props = idx.get("properties") or idx.get("indexedProperties")
            print(name, typ, labels, props)
        except Exception:
            print(idx)


NameError: name 'driver' is not defined

In [26]:
with driver.session() as session:
    result = session.run("""
    MATCH (c:Chunk)
    WHERE c.text CONTAINS "夜盲症"
    RETURN c.id, c.text, c.source
    LIMIT 5
    """)
    for r in result:
        print(r["c.id"], r["c.text"])


0_33 放牧於缺乏微量元素硒的土地上或是攝食品質很差的乾草,造成剛出生仔羊無法站立。

仔羊白肌病的預防方式有這些：1.母羊懷孕期要注意維生素E補充。2.出生仔羊要注意維生素E補充。3.健康的母羊才能生產優質的初乳。4.母羊有胎衣滯留的現象產生。5.仔羊可用注射維生素E的方式補充。6.成羊可以添加維生素E在精料中。

山羊異食癖：仔羊異食癖是源自於礦物質磷的缺乏,磷的缺乏會導致山羊失去食慾,更有產生啃毛、吃皮及雜物等異食癖行為。

山羊異食癖的預防方式有這些：1.日糧內鈣磷比率平衡。2.增加日糧中穀類比例。3.增加日糧中磷酸氫鈣的添加比例。

山羊夜盲症：山羊夜盲症源自於維生素A缺乏,日糧中維生素A缺
0_34 例。3.增加日糧中磷酸氫鈣的添加比例。

山羊夜盲症：山羊夜盲症源自於維生素A缺乏,日糧中維生素A缺乏會導致山羊失去視力。

山羊夜盲症的預防方式有這些：1.注意以放牧飼養山羊的維生素A缺乏。2.無法放牧飼養山羊者需提供青草給飼養的山羊。3.注意在飼料內添加維生素A。4.母羊懷孕期要注意維生素A補充。5.出生仔羊要注意維生素A補充。6.健康的母羊才能生產優質的初乳。

山羊鼓脹(bloat)：山羊鼓脹通常是由大量採食豐富的潮濕新鮮草料或穀類引起的,如三葉草,苜蓿或豆科牧草。這些潮濕新鮮草料在瘤胃中形成微小的氣泡,導致山羊噯氣無法排出。瘤胃隨著泡沫膨脹壓迫橫隔膜,造成山羊可能因呼吸或循環衰竭而很
0_406 之總貯存量。

維生素A之功能包括如下：1.維持正常之視力：因維生素A是眼睛視紫質(rhodopsin)色素形成與再生所必需的物質。所以山羊缺乏維生素A會導致夜盲症，嚴重甚至瞎眼。2.織正常：當缺乏維生素A，則身體之呼吸道、腸道、生殖道、尿道與眼睛表皮細胞會角質化(epithelial keratinization)。3.強抵抗力：因此缺乏維生素A，容易使山羊感染疾病及寄生蟲、繁殖性能不佳。

造成山羊維生素A缺乏之原因有二，主要為飼糧維生素A含量不足，尤其在乾旱季節或冬季，因青草供應受限，或餵飼山羊之乾草品質不良，例如乾草貯存期間太長及乾草貯存不當，因日曬與雨淋。製造乾草時，因天候不良，乾草
0_434 轉換成視覺醇。維生素A在細胞與組織的生長發育上扮演重要的角色。肉羊缺乏維生素A會導致其食慾減少，種公羊會缺乏性慾且精液品質低落。種母羊則會發情不易、生

In [11]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))


In [12]:
# 9️⃣ 驗證：列出部分三元組與來源 (使用關係上的 chunks 陣列與節點 Mentions)
with driver.session() as session:
    result = session.run(
        """
        MATCH (h:Entity)-[r:RELATION]->(t:Entity)
        RETURN h.name AS head, r.type AS relation, t.name AS tail, r.chunks AS chunks
        LIMIT 10
        """
    )
    rows = result.data()
    if not rows:
        print("尚未找到任何三元組，請先執行上一個抽取/寫入 cell。")
    else:
        for row in rows:
            sample = row.get("chunks")
            if isinstance(sample, list) and sample:
                sample = sample[0]
            print(f"({row['head']})-[:{row['relation']}]->({row['tail']})  <- sample chunk: {sample}")

(山羊)-[:每日獲得]->(微生物胺基酸質與量)  <- sample chunk: 0_9
(山羊)-[:受到影響]->(日曬)  <- sample chunk: 0_784
(山羊)-[:易發生]->(波爾種、安格拉種)  <- sample chunk: 0_771
(山羊)-[:調控]->(鈣和有機磷的濃度)  <- sample chunk: 0_769
(山羊)-[:主要原因]->(過度地損失礦物質)  <- sample chunk: 0_768
(山羊)-[:易發生]->(上顎及下顎骨)  <- sample chunk: 0_768
(山羊)-[:罹患]->(纖維性骨營養不良症)  <- sample chunk: 0_767
(山羊)-[:患有]->(纖維性骨營養不良症(Osteodystrophia fibrosa))  <- sample chunk: 0_766
(山羊)-[:死亡]->(巴斯德桿菌性肺炎)  <- sample chunk: 0_746
(山羊)-[:infected_by]->(Mycobacterium paratuberculosis)  <- sample chunk: 0_742


In [18]:
# 9️⃣ 驗證：列出部分三元組與來源 (使用關係上的 chunks 陣列與節點 Mentions)


with driver.session() as session:
    result = session.run(
        """
        MATCH (h:Entity)-[r:RELATION]->(t:Entity)
        WHERE h.name CONTAINS '夜盲症' AND r.chunks <> 'None'
        OR t.name CONTAINS '夜盲症' AND r.chunks <> 'None'
        OR r.chunks CONTAINS '夜盲症' AND r.chunks <> 'None'
        RETURN h.name AS head, r.type AS relation, t.name AS tail, r.chunks AS chunks
        LIMIT 10
        """
    )
    rows = result.data()
    if not rows:
        print("尚未找到任何三元組，請先執行上一個抽取/寫入 cell。")
    else:
        for row in rows:
            sample = row.get("chunks")
            if isinstance(sample, list) and sample:
                sample = sample[0]
            print(f"({row['head']})-[:{row['relation']}]->({row['tail']})  <- sample chunk: {sample}")
            #print(f"({row['head']})-[:{row['relation']}]->({row['tail']}) ")

(山羊)-[:缺乏導致]->(夜盲症)  <- sample chunk: 0_406
(維生素A)-[:缺乏]->(山羊夜盲症)  <- sample chunk: 0_33
(維生素A)-[:缺乏]->(夜盲症)  <- sample chunk: 0_21
(維生素A)-[:缺乏導致]->(夜盲症)  <- sample chunk: 0_17
(肉羊)-[:缺乏]->(夜盲症)  <- sample chunk: 0_434
(維生素E)-[:缺乏導致]->(夜盲症)  <- sample chunk: 0_409
(山羊夜盲症)-[:源自於]->(維生素A缺乏)  <- sample chunk: 0_34
(缺乏維生素A)-[:導致]->(夜盲症)  <- sample chunk: 0_406
(維他命A)-[:缺乏]->(夜盲症)  <- sample chunk: 0_617


In [ ]:
# 一次執行完整流程：建立 Chunk、向量索引、寫入向量、抽取三元組入圖譜、建立檢索與 QA
# 建議在重啟 Kernel 後，直接只執行本 Cell。

import os
import re
import json
import time
from typing import List, Dict

from neo4j import GraphDatabase
from huggingface_hub import snapshot_download

# neo4j-graphrag 相關（確保套件已安裝）
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OllamaLLM

import ollama

# -----------------------------
# 參數設定
# -----------------------------
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PW = "neo4jgoat"

DATA_FOLDER = r"C:\\Users\\aint\\Desktop\\LLM+KB\\neo4j-graphrag-python\\data"
EMBED_MODEL_REPO = "GanymedeNil/text2vec-large-chinese"
EMBED_MODEL_DIR = "./models/text2vec-large-chinese"
OLLAMA_MODEL = "llama3:8b-instruct-q4_K_M"
TOP_K = 5

# -----------------------------
# 連線 Neo4j
# -----------------------------
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))

# -----------------------------
# 工具函式
# -----------------------------
def chunk_text(text: str, max_length: int = 300, overlap: int = 50) -> List[str]:
    chunks = []
    start = 0
    while start < len(text):
        end = start + max_length
        chunks.append(text[start:end])
        start += max_length - overlap
    return chunks


def ensure_vector_index(driver, name: str, label: str, embedding_property: str, dimensions: int, similarity_fn: str = "cosine"):
    with driver.session() as session:
        existing = session.run("SHOW INDEXES").data()
        if any(idx.get("name") == name for idx in existing):
            print(f"✅ Vector index '{name}' 已存在")
            return
        cypher = f"""
        CREATE VECTOR INDEX {name}
        FOR (n:{label}) ON (n.{embedding_property})
        OPTIONS {{
            indexConfig: {{
                `vector.dimensions`: {dimensions},
                `vector.similarity_function`: '{similarity_fn}'
            }}
        }}
        """
        session.run(cypher)
        print(f"🚀 Vector index '{name}' 建立完成 (dim={dimensions}, sim={similarity_fn})")


def load_txt_files(folder: str) -> List[Dict[str, str]]:
    docs = []
    for i, fname in enumerate(sorted(os.listdir(folder))):
        if fname.lower().endswith(".txt"):
            path = os.path.join(folder, fname)
            with open(path, "r", encoding="utf-8") as f:
                text = f.read()
            chunks = chunk_text(text)
            for j, chunk in enumerate(chunks):
                docs.append({
                    "id": f"{i}_{j}",
                    "text": chunk,
                    "source": fname,
                })
    return docs


def ingest_documents(docs: List[Dict[str, str]], embedder: SentenceTransformerEmbeddings, driver) -> None:
    with driver.session() as session:
        for doc in docs:
            emb = embedder.embed_query(doc["text"])
            if hasattr(emb, "tolist"):
                emb = emb.tolist()
            session.run(
                """
                MERGE (c:Chunk {id:$id})
                SET c.text = $text,
                    c.source = $source,
                    c.embedding = $embedding
                """,
                id=doc["id"],
                text=doc["text"],
                source=doc["source"],
                embedding=emb,
            )
    print(f"✅ 已寫入 {len(docs)} 個 chunk 到 Neo4j")


# ---- 三元組抽取與寫入 ----
TRIPLE_PROMPT_TMPL = """
你是一個資訊抽取器。請從下方文字中抽取知識三元組，輸出純 JSON 陣列，不要加解說文字。
每個元素包含 head, relation, tail 三個欄位，例如：
[{{"head":"山羊","relation":"缺乏","tail":"維生素A"}}]

文字：
{chunk}
"""


def _coerce_triple_list(obj) -> List[Dict[str, str]]:
    out = []
    if isinstance(obj, list):
        for it in obj:
            if isinstance(it, dict):
                h, r, t = it.get("head"), it.get("relation"), it.get("tail")
                if all(isinstance(x, str) and x.strip() for x in (h, r, t)):
                    out.append({"head": h.strip(), "relation": r.strip(), "tail": t.strip()})
            elif isinstance(it, (list, tuple)) and len(it) == 3:
                h, r, t = it
                if all(isinstance(x, str) and x.strip() for x in (h, r, t)):
                    out.append({"head": str(h).strip(), "relation": str(r).strip(), "tail": str(t).strip()})
    return out


def parse_triples_from_text(raw: str) -> List[Dict[str, str]]:
    try:
        return _coerce_triple_list(json.loads(raw))
    except Exception:
        m = re.search(r"\[[\s\S]*\]", raw)
        if m:
            try:
                return _coerce_triple_list(json.loads(m.group(0)))
            except Exception:
                return []
        return []


def extract_triples_from_chunk(chunk_text: str, model: str = OLLAMA_MODEL) -> List[Dict[str, str]]:
    prompt = TRIPLE_PROMPT_TMPL.format(chunk=chunk_text)
    resp = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        options={"temperature": 0}
    )
    content = resp.get("message", {}).get("content", "")
    return parse_triples_from_text(content)


def ingest_triples_with_provenance(triples: List[Dict[str, str]], chunk_id: str, driver) -> int:
    if not triples:
        return 0
    with driver.session() as session:
        for t in triples:
            head, rel, tail = t["head"], t["relation"], t["tail"]
            session.run(
                """
                MERGE (h:Entity {name:$head})
                MERGE (t:Entity {name:$tail})
                MERGE (h)-[r:RELATION {type:$rel}]->(t)
                WITH h, t, r, $cid AS cid
                SET r.chunks = CASE
                  WHEN r.chunks IS NULL THEN [cid]
                  WHEN NOT cid IN r.chunks THEN r.chunks + cid
                  ELSE r.chunks
                END
                WITH h, t, r, cid
                MATCH (c:Chunk {id:cid})
                MERGE (c)-[:MENTIONS]->(h)
                MERGE (c)-[:MENTIONS]->(t)
                """,
                head=head,
                rel=rel,
                tail=tail,
                cid=chunk_id,
            )
    return len(triples)


# -----------------------------
# 1) 準備 Embedding 模型與索引
# -----------------------------
if not os.path.isdir(EMBED_MODEL_DIR) or not os.listdir(EMBED_MODEL_DIR):
    print("⏬ 下載中文向量模型……")
    snapshot_download(repo_id=EMBED_MODEL_REPO, local_dir=EMBED_MODEL_DIR)
else:
    print("✅ 已存在本地向量模型")

embedder = SentenceTransformerEmbeddings(model=EMBED_MODEL_DIR)
embedding_dim = embedder.model.get_sentence_embedding_dimension()

ensure_vector_index(driver, name="chunk_embeddings", label="Chunk", embedding_property="embedding", dimensions=embedding_dim, similarity_fn="cosine")

# -----------------------------
# 2) 載入 TXT 並切 Chunk、寫入向量
# -----------------------------
docs = load_txt_files(DATA_FOLDER)
print(f"📄 載入檔案共 {len(docs)} 個 chunk")
ingest_documents(docs, embedder, driver)

# -----------------------------
# 3) 從每個 Chunk 抽取三元組並寫入知識圖譜
# -----------------------------
rel_count = 0
for doc in docs:
    triples = extract_triples_from_chunk(doc["text"], model=OLLAMA_MODEL)
    rel_count += ingest_triples_with_provenance(triples, doc["id"], driver)
print(f"✅ 已寫入 {rel_count} 條 (Entity)-[RELATION]->(Entity) 關係，並建立來源 MENTIONS 與 r.chunks")

# 簡單檢視幾條三元組
with driver.session() as session:
    rows = session.run(
        """
        MATCH (h:Entity)-[r:RELATION]->(t:Entity)
        RETURN h.name AS head, r.type AS relation, t.name AS tail, r.chunks AS chunks
        LIMIT 10
        """
).data()
    for row in rows:
        sample = row.get("chunks")
        if isinstance(sample, list) and sample:
            sample = sample[0]
        print(f"🔎 三元組: ({row['head']})-[:{row['relation']}]->({row['tail']})  <- sample chunk: {sample}")

# -----------------------------
# 4) 建立檢索與 QA（VectorRetriever + Ollama LLM）
# -----------------------------
vector_retriever = VectorRetriever(
    driver=driver,
    index_name="chunk_embeddings",
    embedder=embedder,
    return_properties=["text", "source"],
)
llm = OllamaLLM(model_name=OLLAMA_MODEL)
rag = GraphRAG(llm=llm, retriever=vector_retriever)


# 直接向量查詢 fallback（當 rag 返回中沒有 retriever_result 時）
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings as _STE

def vector_query_fallback(query_text: str, top_k: int = TOP_K):
    q_emb = embedder.embed_query(query_text)
    if hasattr(q_emb, "tolist"):
        q_emb = q_emb.tolist()
    with driver.session() as session:
        rows = session.run(
            """
            CALL db.index.vector.queryNodes('chunk_embeddings', $k, $qemb)
            YIELD node, score
            RETURN node.text AS text, node.source AS source, score
            LIMIT $k
            """,
            k=top_k,
            qemb=q_emb,
        ).data()
    return rows


def _measure_tok_s(prompt: str, model: str = OLLAMA_MODEL) -> float:
    """使用 Ollama 的 eval 統計估算 tok/s；若不可得則嘗試以 eval_count/eval_duration 估算，仍不可得則回傳 -1。"""
    try:
        r = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}], options={"temperature": 0})
        # 1) 直接取 tokens_per_second（若存在）
        eval_info = r.get("eval", {}) or r.get("eval_info", {})
        tps = eval_info.get("tokens_per_second") or eval_info.get("tps")
        if isinstance(tps, (int, float)) and tps > 0:
            return float(tps)
        # 2) 以 eval_count / eval_duration(秒) 推估（Ollama 一般回傳 ns）
        eval_count = r.get("eval_count") or r.get("eval_token_count") or r.get("eval_tokens")
        eval_duration = r.get("eval_duration")  # nanoseconds
        if isinstance(eval_count, (int, float)) and isinstance(eval_duration, (int, float)) and eval_duration:
            seconds = eval_duration / 1e9 if eval_duration > 1e6 else float(eval_duration)
            if seconds > 0:
                return float(eval_count) / seconds
    except Exception:
        pass
    return -1.0

def qa(query: str, top_k: int = TOP_K):
    t0 = time.perf_counter()
    resp = rag.search(query_text=query, retriever_config={"top_k": top_k})
    t1 = time.perf_counter()
    latency_ms = (t1 - t0) * 1000.0
    print("\n❓ 問題:", query)
    print("🟩 回答:", getattr(resp, "answer", None))
    print(f"⏱️ Inference latency: {latency_ms:.1f} ms")
    # 嘗試用最簡 prompt 估計 tok/s（不影響實際回答）
    ctx_nodes = getattr(resp, "retriever_result", None) or []
    ctx_text = "\n\n".join([n.get("text", "") for n in ctx_nodes[:min(len(ctx_nodes), top_k)]])
    probe_prompt = f"請根據以下內容簡短回答：\n\n{ctx_text}\n\n問題：{query}\n請用繁體中文回答。"
    tps = _measure_tok_s(probe_prompt, model=OLLAMA_MODEL)
    if tps > 0:
        print(f"🚀 LLM throughput: {tps:.2f} tok/s (Ollama eval)")
    else:
        print("🚀 LLM throughput: N/A (Ollama 未提供 eval 統計)")
    nodes = getattr(resp, "retriever_result", None)
    if nodes:
        print("🟦 相關內容 (前幾筆):")
        for node in nodes[:min(len(nodes), top_k)]:
            print("-", node.get("text", ""), f"(source={node.get('source')})")
    else:
        print("🟨 檢索結果為空，改用向量索引 fallback：")
        rows = vector_query_fallback(query, top_k)
        if rows:
            for r in rows:
                print("-", r.get("text", ""), f"(source={r.get('source')}) score={r.get('score'):.4f}")
        else:
            print("（查無相關內容）")

# Demo 問題（可自行開關，避免重複輸出）
RUN_DEMOS = True
if RUN_DEMOS:
    qa("山羊缺乏維生素A會導致視力出現什麼問題", top_k=5)

c:\Users\aint\Desktop\LLM+KB\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 已存在本地向量模型


No sentence-transformers model found with name ./models/text2vec-large-chinese. Creating a new one with mean pooling.


✅ Vector index 'chunk_embeddings' 已存在
📄 載入檔案共 799 個 chunk
✅ 已寫入 799 個 chunk 到 Neo4j
✅ 已寫入 799 個 chunk 到 Neo4j
✅ 已寫入 7255 條 (Entity)-[RELATION]->(Entity) 關係，並建立來源 MENTIONS 與 r.chunks
🔎 三元組: (山羊)-[:每日獲得]->(微生物胺基酸質與量)  <- sample chunk: 0_9
🔎 三元組: (山羊)-[:受到影響]->(日曬)  <- sample chunk: 0_784
🔎 三元組: (山羊)-[:易發生]->(波爾種、安格拉種)  <- sample chunk: 0_771
🔎 三元組: (山羊)-[:調控]->(鈣和有機磷的濃度)  <- sample chunk: 0_769
🔎 三元組: (山羊)-[:主要原因]->(過度地損失礦物質)  <- sample chunk: 0_768
🔎 三元組: (山羊)-[:易發生]->(上顎及下顎骨)  <- sample chunk: 0_768
🔎 三元組: (山羊)-[:罹患]->(纖維性骨營養不良症)  <- sample chunk: 0_767
🔎 三元組: (山羊)-[:患有]->(纖維性骨營養不良症(Osteodystrophia fibrosa))  <- sample chunk: 0_766
🔎 三元組: (山羊)-[:死亡]->(巴斯德桿菌性肺炎)  <- sample chunk: 0_746
🔎 三元組: (山羊)-[:infected_by]->(Mycobacterium paratuberculosis)  <- sample chunk: 0_742
✅ 已寫入 7255 條 (Entity)-[RELATION]->(Entity) 關係，並建立來源 MENTIONS 與 r.chunks
🔎 三元組: (山羊)-[:每日獲得]->(微生物胺基酸質與量)  <- sample chunk: 0_9
🔎 三元組: (山羊)-[:受到影響]->(日曬)  <- sample chunk: 0_784
🔎 三元組: (山羊)-[:易發生]->(波爾種、安格拉種)  <- sample ch

In [1]:
RUN_DEMOS = True
if RUN_DEMOS:
    qa("山羊缺乏維生素A會導致視力出現什麼問題？", top_k=5)


NameError: name 'qa' is not defined

> 提醒：為避免重複輸出與重複寫入，建議先 Kernel → Restart，然後只執行最下方「一次執行完整流程」的 Cell。上面的範例程式已改為需手動把 `RUN_DEMOS = True` 才會執行。

In [4]:
# ⚡ QA-only：只建立檢索與QA，不重新寫入或抽取
import os
import time
import ollama
from neo4j import GraphDatabase
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OllamaLLM

# 與完整流程保持一致的設定
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PW = "neo4jgoat"
EMBED_MODEL_DIR = "./models/text2vec-large-chinese"
OLLAMA_MODEL = "llama3:8b-instruct-q4_K_M"
INDEX_NAME = "chunk_embeddings"
TOP_K = 5

# 連線 Neo4j
_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))

# 載入本地 Embedding 模型（不下載、不寫入）
_embedder = SentenceTransformerEmbeddings(model=EMBED_MODEL_DIR)

# 建立檢索與 LLM（不重建索引）
_vector_retriever = VectorRetriever(
    driver=_driver,
    index_name=INDEX_NAME,
    embedder=_embedder,
    return_properties=["text", "source"],
)
_llm = OllamaLLM(model_name=OLLAMA_MODEL)
_rag = GraphRAG(llm=_llm, retriever=_vector_retriever)

# 向量查詢 fallback（若 retriever_result 為空）
def _vector_query_fallback(query_text: str, top_k: int = TOP_K):
    q_emb = _embedder.embed_query(query_text)
    if hasattr(q_emb, "tolist"):
        q_emb = q_emb.tolist()
    with _driver.session() as session:
        rows = session.run(
            """
            CALL db.index.vector.queryNodes($index, $k, $qemb)
            YIELD node, score
            RETURN node.text AS text, node.source AS source, score
            LIMIT $k
            """,
            index=INDEX_NAME,
            k=top_k,
            qemb=q_emb,
        ).data()
    return rows

def _measure_tok_s(prompt: str, model: str = OLLAMA_MODEL) -> float:
    try:
        r = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}], options={"temperature": 0})
        # 1) 直接從 eval 拿 tokens_per_second
        eval_info = r.get("eval", {}) or r.get("eval_info", {})
        tps = eval_info.get("tokens_per_second") or eval_info.get("tps")
        if isinstance(tps, (int, float)) and tps > 0:
            return float(tps)
        # 2) 以 eval_count / eval_duration 推估（eval_duration 通常為奈秒）
        eval_count = r.get("eval_count") or r.get("eval_token_count") or r.get("eval_tokens")
        eval_duration = r.get("eval_duration")
        if isinstance(eval_count, (int, float)) and isinstance(eval_duration, (int, float)) and eval_duration:
            seconds = eval_duration / 1e9 if eval_duration > 1e6 else float(eval_duration)
            if seconds > 0:
                return float(eval_count) / seconds
    except Exception:
        pass
    return -1.0

# 封裝成可重用的 QA 函式
def qa_only(query: str, top_k: int = TOP_K, temperature: float | None = None):
    # 根據 temperature 動態建立 LLM 與 RAG（不影響預設 _rag）
    rag_to_use = _rag
    if temperature is not None:
        try:
            _llm_temp = OllamaLLM(
                model_name=OLLAMA_MODEL,
                model_params={"options": {"temperature": float(temperature)}},
            )
            rag_to_use = GraphRAG(llm=_llm_temp, retriever=_vector_retriever)
        except Exception as e:
            print(f"[QA-only] 建立帶 temperature 的 LLM 失敗，改用預設：{e}")
            rag_to_use = _rag

    t0 = time.perf_counter()
    resp = rag_to_use.search(query_text=query, retriever_config={"top_k": top_k})
    t1 = time.perf_counter()
    latency_ms = (t1 - t0) * 1000.0
    print("\n問題:", query)
    print("回答:", getattr(resp, "answer", None))
    print(f"Inference latency: {latency_ms:.1f} ms")
    nodes = getattr(resp, "retriever_result", None)
    # 嘗試用最簡 prompt 估計 tok/s（不影響實際回答）
    ctx_text = "\n\n".join([n.get("text", "") for n in (nodes or [])[:min(len(nodes or []), top_k)]])
    probe_prompt = f"請根據以下內容簡短回答：\n\n{ctx_text}\n\n問題：{query}\n請用繁體中文回答。"
    tps = _measure_tok_s(probe_prompt, model=OLLAMA_MODEL)
    if tps > 0:
        print(f"LLM throughput: {tps:.2f} tok/s (Ollama eval)")
    else:
        print("LLM throughput: N/A (無 eval 統計)")
    if nodes:
        print("相關內容 (前幾筆):")
        for node in nodes[:min(len(nodes), top_k)]:
            print("-", node.get("text", ""), f"(source={node.get('source')})")
    else:
        print("檢索結果為空，改用向量索引 fallback：")
        rows = _vector_query_fallback(query, top_k)
        if rows:
            for r in rows:
                try:
                    score = r.get("score")
                    score_s = f" score={score:.4f}" if isinstance(score, (int, float)) else ""
                except Exception:
                    score_s = ""
                print("-", r.get("text", ""), f"(source={r.get('source')})" + score_s)
        else:
            print("（查無相關內容）")

# 範例（預設不自動執行）
RUN_QA_ONLY_DEMO = True
if RUN_QA_ONLY_DEMO:
    qa_only("山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答", top_k=5, temperature=0.7)

No sentence-transformers model found with name ./models/text2vec-large-chinese. Creating a new one with mean pooling.



問題: 山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答
回答: According to the provided context, 山羊缺乏維生素A會導致夜盲症，嚴重甚至瞎眼。
Inference latency: 4151.0 ms
LLM throughput: 85.02 tok/s (Ollama eval)
檢索結果為空，改用向量索引 fallback：
- 生素群,以滿足山羊生理需要。

維生素A:維生素A的缺乏會導致山羊食慾減少,種公羊精液品質降低、缺乏性慾,母羊則發情不易、生殖效率降低,嚴重缺乏亦會造成山羊夜盲之病變。因此1981年版的NRC,建議維生素A在山羊每公斤日糧中至少含3500國際單位(IU)以上。而2007年版的NRC則將山羊維生素A需要量改以視網醇當量(retinol equivalent,RE)為估算單位。

維生素D:維生素D在山羊的鈣、磷吸收上扮演著非常重要的角色,缺乏維生素D會導致山羊鈣磷吸收不平衡、影響山羊骨骼發育。放牧羊隻會因太陽照射其皮膚而自行合成維生素D,故不會缺乏維生素D。然而圈飼羊隻則因無法長時間接觸太陽光與 (source=read.txt) score=0.8935
- 之總貯存量。

維生素A之功能包括如下：1.維持正常之視力：因維生素A是眼睛視紫質(rhodopsin)色素形成與再生所必需的物質。所以山羊缺乏維生素A會導致夜盲症，嚴重甚至瞎眼。2.織正常：當缺乏維生素A，則身體之呼吸道、腸道、生殖道、尿道與眼睛表皮細胞會角質化(epithelial keratinization)。3.強抵抗力：因此缺乏維生素A，容易使山羊感染疾病及寄生蟲、繁殖性能不佳。

造成山羊維生素A缺乏之原因有二，主要為飼糧維生素A含量不足，尤其在乾旱季節或冬季，因青草供應受限，或餵飼山羊之乾草品質不良，例如乾草貯存期間太長及乾草貯存不當，因日曬與雨淋。製造乾草時，因天候不良，乾草 (source=read.txt) score=0.8598
- 轉換成視覺醇。維生素A在細胞與組織的生長發育上扮演重要的角色。肉羊缺乏維生素A會導致其食慾減少，種公羊會缺乏性慾且精液品質低落。種母羊則會發情不易、生殖效率差，更嚴重的缺乏會造成肉羊眼睛病變而形成夜盲症。因此在肉羊日糧中，每公斤精料至少要含3500國際單位(IU)之維生素A。

維

In [19]:
RUN_QA_ONLY_DEMO = True
if RUN_QA_ONLY_DEMO:
    qa_only("山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答", top_k=5, temperature=0.7)


問題: 山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答
回答: According to the provided context, mountain goats deficient in vitamin A may lead to:

* 夜盲症 (night blindness) due to a lack of vision
* 角質化 (keratinization) of epithelial cells in the respiratory tract, intestines, reproductive organs, and skin
* 弱抵抗力 (weak immune system), making them more susceptible to diseases and parasites

Additionally, severe vitamin A deficiency can cause 山羊夜盲症 (mountain goat night blindness), which is characterized by a lack of vision.
Inference latency: 7035.8 ms
LLM throughput: 88.86 tok/s (Ollama eval)
檢索結果為空，改用向量索引 fallback：
- 生素群,以滿足山羊生理需要。

維生素A:維生素A的缺乏會導致山羊食慾減少,種公羊精液品質降低、缺乏性慾,母羊則發情不易、生殖效率降低,嚴重缺乏亦會造成山羊夜盲之病變。因此1981年版的NRC,建議維生素A在山羊每公斤日糧中至少含3500國際單位(IU)以上。而2007年版的NRC則將山羊維生素A需要量改以視網醇當量(retinol equivalent,RE)為估算單位。

維生素D:維生素D在山羊的鈣、磷吸收上扮演著非常重要的角色,缺乏維生素D會導致山羊鈣磷吸收不平衡、影響山羊骨骼發育。放牧羊隻會因太陽照射其皮膚而自行合成維生素D,故不會缺乏維生素D。然而圈飼羊隻則因無法長時間接觸太陽光與 (source=read.txt) score=0.8935
- 之總貯存量。

維生素A之功能包括如下：1.維持正常之視力：因維生素A是眼睛視紫質(rhodopsin)色素形成與再生所必需的物質。所

### 如何使用 QA-only 模式

- 不想重跑整個流程時，直接執行上一個「QA-only」程式碼 Cell。
- 該 Cell 只連線 Neo4j、載入本地向量模型、建立檢索與 LLM，不會重建索引或重寫資料。
- 呼叫方式：
  - `qa_only("你的問題", top_k=5)`
- 如需快速示範，把 `RUN_QA_ONLY_DEMO = True` 後再次執行該 Cell。

#### 額外輸出說明（Latency 與 tok/s）
- ⏱️ Inference latency: 包含檢索與 LLM 生成的端到端時間（毫秒）。
- 🚀 LLM throughput (tok/s): 透過一次極簡 `ollama.chat` 取得 Ollama 的 eval 統計（若版本/模型不支援則顯示 N/A）。這不會影響實際回答，只作為參考效能指標。

## ✅ Schema hardening and indexes (optional)

這個區塊會：
- 新增唯一性限制：`Chunk.id`、`Entity.name`
- 建立 Fulltext 索引：`chunk_text_fts` 於 `Chunk(text)`，支援混合檢索

執行一次即可（之後會自動跳過已存在的索引/限制）。

In [4]:
# 建立唯一性限制與 Fulltext 索引（若不存在）
from neo4j import GraphDatabase

_uri = "bolt://localhost:7687"
_user = "neo4j"
_pw = "neo4jgoat"

_driver = GraphDatabase.driver(_uri, auth=(_user, _pw))
with _driver.session() as session:
    # constraints
    session.run("CREATE CONSTRAINT chunk_id_unique IF NOT EXISTS FOR (c:Chunk) REQUIRE c.id IS UNIQUE")
    session.run("CREATE CONSTRAINT entity_name_unique IF NOT EXISTS FOR (e:Entity) REQUIRE e.name IS UNIQUE")
    # fulltext
    session.run(
        """
        CREATE FULLTEXT INDEX chunk_text_fts IF NOT EXISTS
        FOR (c:Chunk) ON EACH [c.text]
        """
    )
print("Schema and fulltext index ensured ✅")

Schema and fulltext index ensured ✅


## 🔎 Hybrid retrieval and graph expansion (optional)

這個區塊示範：
- 使用 `HybridRetriever` 同時結合向量索引與 Fulltext 索引（可選線性加權）
- 以檢索到的 Chunk 擴展到其 `MENTIONS` 的 `Entity` 與 `RELATION` 三元組，組合更豐富的上下文

你可以與現有 `VectorRetriever + GraphRAG` 併存，互不影響。

# HybridRetriever + Graph expansion QA
from neo4j_graphrag.retrievers.hybrid import HybridRetriever, HybridSearchRanker
from neo4j_graphrag.retrievers import VectorRetriever as _VectorRetriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OllamaLLM
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings
from neo4j import GraphDatabase
import time
import ollama

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PW = "neo4jgoat"
INDEX_VECTOR = "chunk_embeddings"
INDEX_FTS = "chunk_text_fts"
EMBED_MODEL_DIR = "./models/text2vec-large-chinese"
OLLAMA_MODEL = "llama3:8b-instruct-q4_K_M"

_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))
_embedder = SentenceTransformerEmbeddings(model=EMBED_MODEL_DIR)

# Ensure the fulltext index exists and is online (idempotent, with procedure fallback)
def _ensure_fulltext_index(driver, index_name: str, label: str, prop: str = "text", strict: bool = False) -> bool:
    """Try to ensure a fulltext index exists. Returns True if exists/created, else False.
    If strict=True, raises when cannot be ensured.
    """
    def _exists(sess) -> bool:
        try:
            rows = sess.run("SHOW INDEXES").data()
            if any(str(r.get("name")) == index_name and (r.get("type") == "FULLTEXT" or r.get("indexType") == "FULLTEXT") for r in rows):
                return True
        except Exception:
            pass
        # Fallback check via procedure
        try:
            r = sess.run("CALL db.index.fulltext.list() YIELD name WHERE name = $name RETURN name", name=index_name).single()
            if r:
                return True
        except Exception:
            pass
        return False

    with driver.session() as session:
        if _exists(session):
            return True
        created = False
        # Try schema DDL (Neo4j 5.x)
        try:
            cypher = f"""
            CREATE FULLTEXT INDEX {index_name} IF NOT EXISTS
            FOR (n:{label}) ON EACH [n.{prop}]
            """
            session.run(cypher)
            created = True
        except Exception:
            created = False
        # Fallback to procedure (Neo4j 4.x/compat)
        if not created:
            try:
                session.run(
                    "CALL db.index.fulltext.createNodeIndex($name, $labels, $props)",
                    name=index_name,
                    labels=[label],
                    props=[prop],
                )
                created = True
            except Exception:
                created = False
        # Wait until online (best-effort)
        try:
            session.run("CALL db.awaitIndexes()")
        except Exception:
            pass
        ok = _exists(session)
        if not ok and strict:
            raise RuntimeError(f"無法建立 Fulltext 索引: {index_name}")
        return ok

_USING_VECTOR_ONLY = False
_has_fts = _ensure_fulltext_index(_driver, INDEX_FTS, label="Chunk", prop="text", strict=False)

# 1) 準備檢索器：若 Fulltext 不可用，則退化成向量-only 檢索
if _has_fts:
    _hybrid = HybridRetriever(
        driver=_driver,
        vector_index_name=INDEX_VECTOR,
        fulltext_index_name=INDEX_FTS,
        embedder=_embedder,
        return_properties=["text", "source", "id"],
    )
else:
    print("⚠️ Fulltext 索引不可用或無法建立，將使用向量-only 檢索做為替代。")
    _USING_VECTOR_ONLY = True
    _hybrid = _VectorRetriever(
        driver=_driver,
        index_name=INDEX_VECTOR,
        embedder=_embedder,
        return_properties=["text", "source", "id"],
    )

_llm = OllamaLLM(model_name=OLLAMA_MODEL)
_rag_hybrid = GraphRAG(retriever=_hybrid, llm=_llm)


def _measure_tok_s_hybrid(prompt: str, model: str = OLLAMA_MODEL) -> float:
    """Estimate tok/s from Ollama eval stats; return -1.0 if unavailable."""
    try:
        r = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}], options={"temperature": 0})
        eval_info = r.get("eval", {}) or r.get("eval_info", {})
        tps = eval_info.get("tokens_per_second") or r.get("tps")
        if isinstance(tps, (int, float)) and tps > 0:
            return float(tps)
        eval_count = r.get("eval_count") or r.get("eval_token_count") or r.get("eval_tokens")
        eval_duration = r.get("eval_duration")  # nanoseconds
        if isinstance(eval_count, (int, float)) and isinstance(eval_duration, (int, float)) and eval_duration:
            seconds = eval_duration / 1e9 if eval_duration > 1e6 else float(eval_duration)
            if seconds > 0:
                return float(eval_count) / seconds
    except Exception:
        pass
    return -1.0


def qa_hybrid(query: str, top_k: int = 5, ranker: str = "naive", alpha: float | None = None, temperature: float | None = None):
    """Hybrid QA with optional temperature to control LLM variability."""
    # choose LLM per call if temperature is provided
    rag_to_use = _rag_hybrid
    if temperature is not None:
        try:
            _llm_tmp = OllamaLLM(
                model_name=OLLAMA_MODEL,
                model_params={"options": {"temperature": float(temperature)}},
            )
            rag_to_use = GraphRAG(retriever=_hybrid, llm=_llm_tmp)
        except Exception as e:
            print(f"[Hybrid] 建立帶 temperature 的 LLM 失敗，改用預設：{e}")
            rag_to_use = _rag_hybrid

    if _USING_VECTOR_ONLY:
        print("[Hybrid] 提示：Fulltext 不可用，現以向量-only 模式執行。")
    cfg = {"top_k": top_k}
    if not _USING_VECTOR_ONLY and ranker == "linear":
        cfg["ranker"] = HybridSearchRanker.LINEAR
        cfg["alpha"] = alpha if alpha is not None else 0.6
    t0 = time.perf_counter()
    resp = rag_to_use.search(query_text=query, retriever_config=cfg, return_context=True)
    t1 = time.perf_counter()
    latency_ms = (t1 - t0) * 1000.0

    print("\n[Hybrid] 問題:", query)
    print("[Hybrid] 回答:", resp.answer)

    # Print latency and estimate tok/s via a small probe
    print(f"Inference latency: {latency_ms:.1f} ms")
    items = getattr(resp, "retriever_result", None)
    # Build a small context snippet for throughput probe
    ctx_texts = []
    if items:
        try:
            # Hybrid items container
            for it in items.items[:min(len(items.items), top_k)]:
                content = getattr(it, "content", None)
                if not content and isinstance(getattr(it, "metadata", None), dict):
                    content = getattr(it, "metadata", {}).get("text")
                if content:
                    ctx_texts.append(str(content))
        except Exception:
            # Vector-only list of dicts
            try:
                for n in items[:min(len(items), top_k)]:
                    ctx_texts.append(str(n.get("text", "")))
            except Exception:
                pass
    ctx_blob = "\n\n".join(ctx_texts)
    probe_prompt = f"請根據以下內容簡短回答：\n\n{ctx_blob}\n\n問題：{query}\n請用繁體中文回答。"
    tps = _measure_tok_s_hybrid(probe_prompt, model=OLLAMA_MODEL)
    if tps > 0:
        print(f"LLM throughput: {tps:.2f} tok/s (Ollama eval)")
    else:
        print("LLM throughput: N/A (無 eval 統計)")

    if items:
        print("[Hybrid] 檢索結果摘要:")
        # 支援 HybridRetriever 的 items 容器或 VectorRetriever 的 list 節點
        try:
            # Hybrid path
            for it in items.items[:min(len(items.items), top_k)]:
                content = getattr(it, "content", None) or it.get("text", "")
                meta = getattr(it, "metadata", None) or it.get("source", "")
                print("-", str(content)[:120].replace("\n", " "), str(meta)[:80])
        except Exception:
            # Vector-only path (list of dicts)
            try:
                for n in items[:min(len(items), top_k)]:
                    print("-", str(n.get("text", ""))[:120].replace("\n", " "), f"(source={n.get('source')})")
            except Exception:
                pass

# 2) 從檢索到的 Chunk 擴展圖（MENTIONS 與 RELATION）

def expand_context_from_chunks(query: str, top_k: int = 5) -> str:
    q_emb = _embedder.embed_query(query)
    if hasattr(q_emb, "tolist"):
        q_emb = q_emb.tolist()
    cypher = """
    CALL db.index.vector.queryNodes($index, $k, $qemb)
    YIELD node, score
    WITH collect(node)[..$k] AS chunks
    // 擴展相關實體與三元組
    UNWIND chunks AS c
    OPTIONAL MATCH (c)-[:MENTIONS]->(e:Entity)
    OPTIONAL MATCH (e)<-[r:RELATION]->(t:Entity)
    WITH chunks, collect(DISTINCT e.name) AS ents,
         collect(DISTINCT {h:e.name, rel:r.type, t:t.name, sample: head(coalesce(r.chunks, []))}) AS triples
    RETURN [c IN chunks | c.text][..$k] AS chunk_texts, ents[..50] AS entities, triples[..100] AS triples
    """
    with _driver.session() as session:
        row = session.run(cypher, index=INDEX_VECTOR, k=top_k, qemb=q_emb).single()
    if not row:
        return ""
    chunks = row.get("chunk_texts") or []
    ents = row.get("entities") or []
    triples = row.get("triples") or []
    triple_lines = [f"({t.get('h')})-[:{t.get('rel')}]->({t.get('t')})" for t in triples if t]
    context = (
        "[Chunks]\n" + "\n\n".join(chunks) +
        "\n\n[Entities]\n" + ", ".join(ents) +
        "\n\n[Triples]\n" + "\n".join(triple_lines)
    )
    return context


def qa_graph(query: str, top_k: int = 5, temperature: float | None = None):
    """Graph expansion QA with metrics and adjustable temperature.

    Args:
        query: 問題
        top_k: 檢索擴展的 chunk 數量
        temperature: 若提供，將以該值建立臨時 LLM 以控制隨機性（0 更保守，1 更發散）。
    """
    t0 = time.perf_counter()
    context = expand_context_from_chunks(query, top_k=top_k)
    if not context:
        print("[GraphExpand] 無檢索結果")
        return

    prompt = f"請根據提供的內容回答問題，條列且含依據。\n\n{context}\n\n問題：{query}\n請用繁體中文回答。"

    # 選擇 LLM（支援 per-call 調整 temperature）
    llm_obj = _llm
    if temperature is not None:
        try:
            llm_obj = OllamaLLM(
                model_name=OLLAMA_MODEL,
                model_params={"options": {"temperature": float(temperature)}},
            )
        except Exception as e:
            print(f"[GraphExpand] 建立帶 temperature 的 LLM 失敗，改用預設：{e}")
            llm_obj = _llm

    ans = llm_obj.invoke(prompt).content
    t1 = time.perf_counter()
    latency_ms = (t1 - t0) * 1000.0

    print("\n[GraphExpand] 問題:", query)
    print("[GraphExpand] 回答:\n", ans)
    print(f"Inference latency: {latency_ms:.1f} ms")

    # 以相同 context 構造極簡 probe 估算 tok/s
    probe_prompt = f"請根據以下內容簡短回答：\n\n{context}\n\n問題：{query}\n請用繁體中文回答。"
    tps = _measure_tok_s_hybrid(probe_prompt, model=OLLAMA_MODEL)
    if tps > 0:
        print(f"LLM throughput: {tps:.2f} tok/s (Ollama eval)")
    else:
        print("LLM throughput: N/A (無 eval 統計)")

In [1]:
# HybridRetriever + Graph expansion QA
from neo4j_graphrag.retrievers.hybrid import HybridRetriever, HybridSearchRanker
from neo4j_graphrag.retrievers import VectorRetriever as _VectorRetriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OllamaLLM
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings
from neo4j import GraphDatabase
import time
import ollama

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PW = "neo4jgoat"
INDEX_VECTOR = "chunk_embeddings"
INDEX_FTS = "chunk_text_fts"
EMBED_MODEL_DIR = "./models/text2vec-large-chinese"
OLLAMA_MODEL = "llama3:8b-instruct-q4_K_M"

_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))
_embedder = SentenceTransformerEmbeddings(model=EMBED_MODEL_DIR)

# Ensure the fulltext index exists and is online (idempotent, with procedure fallback)
def _ensure_fulltext_index(driver, index_name: str, label: str, prop: str = "text", strict: bool = False) -> bool:
    """Try to ensure a fulltext index exists. Returns True if exists/created, else False.
    If strict=True, raises when cannot be ensured.
    """
    def _exists(sess) -> bool:
        try:
            rows = sess.run("SHOW INDEXES").data()
            if any(str(r.get("name")) == index_name and (r.get("type") == "FULLTEXT" or r.get("indexType") == "FULLTEXT") for r in rows):
                return True
        except Exception:
            pass
        # Fallback check via procedure
        try:
            r = sess.run("CALL db.index.fulltext.list() YIELD name WHERE name = $name RETURN name", name=index_name).single()
            if r:
                return True
        except Exception:
            pass
        return False

    with driver.session() as session:
        if _exists(session):
            return True
        created = False
        # Try schema DDL (Neo4j 5.x)
        try:
            cypher = f"""
            CREATE FULLTEXT INDEX {index_name} IF NOT EXISTS
            FOR (n:{label}) ON EACH [n.{prop}]
            """
            session.run(cypher)
            created = True
        except Exception:
            created = False
        # Fallback to procedure (Neo4j 4.x/compat)
        if not created:
            try:
                session.run(
                    "CALL db.index.fulltext.createNodeIndex($name, $labels, $props)",
                    name=index_name,
                    labels=[label],
                    props=[prop],
                )
                created = True
            except Exception:
                created = False
        # Wait until online (best-effort)
        try:
            session.run("CALL db.awaitIndexes()")
        except Exception:
            pass
        ok = _exists(session)
        if not ok and strict:
            raise RuntimeError(f"無法建立 Fulltext 索引: {index_name}")
        return ok

_USING_VECTOR_ONLY = False
_has_fts = _ensure_fulltext_index(_driver, INDEX_FTS, label="Chunk", prop="text", strict=False)

# 1) 準備檢索器：若 Fulltext 不可用，則退化成向量-only 檢索
if _has_fts:
    _hybrid = HybridRetriever(
        driver=_driver,
        vector_index_name=INDEX_VECTOR,
        fulltext_index_name=INDEX_FTS,
        embedder=_embedder,
        return_properties=["text", "source", "id"],
    )
else:
    print("⚠️ Fulltext 索引不可用或無法建立，將使用向量-only 檢索做為替代。")
    _USING_VECTOR_ONLY = True
    _hybrid = _VectorRetriever(
        driver=_driver,
        index_name=INDEX_VECTOR,
        embedder=_embedder,
        return_properties=["text", "source", "id"],
    )

_llm = OllamaLLM(model_name=OLLAMA_MODEL)
_rag_hybrid = GraphRAG(retriever=_hybrid, llm=_llm)


def _measure_tok_s_hybrid(prompt: str, model: str = OLLAMA_MODEL) -> float:
    """Estimate tok/s from Ollama eval stats; return -1.0 if unavailable."""
    try:
        r = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}], options={"temperature": 0})
        eval_info = r.get("eval", {}) or r.get("eval_info", {})
        tps = eval_info.get("tokens_per_second") or r.get("tps")
        if isinstance(tps, (int, float)) and tps > 0:
            return float(tps)
        eval_count = r.get("eval_count") or r.get("eval_token_count") or r.get("eval_tokens")
        eval_duration = r.get("eval_duration")  # nanoseconds
        if isinstance(eval_count, (int, float)) and isinstance(eval_duration, (int, float)) and eval_duration:
            seconds = eval_duration / 1e9 if eval_duration > 1e6 else float(eval_duration)
            if seconds > 0:
                return float(eval_count) / seconds
    except Exception:
        pass
    return -1.0


def qa_hybrid(query: str, top_k: int = 5, ranker: str = "naive", alpha: float | None = None, temperature: float | None = None):
    """Hybrid QA with optional temperature to control LLM variability."""
    # choose LLM per call if temperature is provided
    rag_to_use = _rag_hybrid
    if temperature is not None:
        try:
            _llm_tmp = OllamaLLM(
                model_name=OLLAMA_MODEL,
                model_params={"options": {"temperature": float(temperature)}},
            )
            rag_to_use = GraphRAG(retriever=_hybrid, llm=_llm_tmp)
        except Exception as e:
            print(f"[Hybrid] 建立帶 temperature 的 LLM 失敗，改用預設：{e}")
            rag_to_use = _rag_hybrid

    if _USING_VECTOR_ONLY:
        print("[Hybrid] 提示：Fulltext 不可用，現以向量-only 模式執行。")
    cfg = {"top_k": top_k}
    if not _USING_VECTOR_ONLY and ranker == "linear":
        cfg["ranker"] = HybridSearchRanker.LINEAR
        cfg["alpha"] = alpha if alpha is not None else 0.6
    t0 = time.perf_counter()
    resp = rag_to_use.search(query_text=query, retriever_config=cfg, return_context=True)
    t1 = time.perf_counter()
    latency_ms = (t1 - t0) * 1000.0

    print("\n[Hybrid] 問題:", query)
    print("[Hybrid] 回答:", resp.answer)

    # Print latency and estimate tok/s via a small probe
    print(f"Inference latency: {latency_ms:.1f} ms")
    items = getattr(resp, "retriever_result", None)
    # Build a small context snippet for throughput probe
    ctx_texts = []
    if items:
        try:
            # Hybrid items container
            for it in items.items[:min(len(items.items), top_k)]:
                content = getattr(it, "content", None)
                if not content and isinstance(getattr(it, "metadata", None), dict):
                    content = getattr(it, "metadata", {}).get("text")
                if content:
                    ctx_texts.append(str(content))
        except Exception:
            # Vector-only list of dicts
            try:
                for n in items[:min(len(items), top_k)]:
                    ctx_texts.append(str(n.get("text", "")))
            except Exception:
                pass
    ctx_blob = "\n\n".join(ctx_texts)
    probe_prompt = f"請根據以下內容簡短回答：\n\n{ctx_blob}\n\n問題：{query}\n請用繁體中文回答。"
    tps = _measure_tok_s_hybrid(probe_prompt, model=OLLAMA_MODEL)
    if tps > 0:
        print(f"LLM throughput: {tps:.2f} tok/s (Ollama eval)")
    else:
        print("LLM throughput: N/A (無 eval 統計)")

    if items:
        print("[Hybrid] 檢索結果摘要:")
        # 支援 HybridRetriever 的 items 容器或 VectorRetriever 的 list 節點
        try:
            # Hybrid path
            for it in items.items[:min(len(items.items), top_k)]:
                content = getattr(it, "content", None) or it.get("text", "")
                meta = getattr(it, "metadata", None) or it.get("source", "")
                print("-", str(content)[:120].replace("\n", " "), str(meta)[:80])
        except Exception:
            # Vector-only path (list of dicts)
            try:
                for n in items[:min(len(items), top_k)]:
                    print("-", str(n.get("text", ""))[:120].replace("\n", " "), f"(source={n.get('source')})")
            except Exception:
                pass

# 2) 從檢索到的 Chunk 擴展圖（MENTIONS 與 RELATION）

def expand_context_from_chunks(query: str, top_k: int = 5) -> str:
    q_emb = _embedder.embed_query(query)
    if hasattr(q_emb, "tolist"):
        q_emb = q_emb.tolist()
    cypher = """
    CALL db.index.vector.queryNodes($index, $k, $qemb)
    YIELD node, score
    WITH collect(node)[..$k] AS chunks
    // 擴展相關實體與三元組
    UNWIND chunks AS c
    OPTIONAL MATCH (c)-[:MENTIONS]->(e:Entity)
    OPTIONAL MATCH (e)<-[r:RELATION]->(t:Entity)
    WITH chunks, collect(DISTINCT e.name) AS ents,
         collect(DISTINCT {h:e.name, rel:r.type, t:t.name, sample: head(coalesce(r.chunks, []))}) AS triples
    RETURN [c IN chunks | c.text][..$k] AS chunk_texts, ents[..50] AS entities, triples[..100] AS triples
    """
    with _driver.session() as session:
        row = session.run(cypher, index=INDEX_VECTOR, k=top_k, qemb=q_emb).single()
    if not row:
        return ""
    chunks = row.get("chunk_texts") or []
    ents = row.get("entities") or []
    triples = row.get("triples") or []
    triple_lines = [f"({t.get('h')})-[:{t.get('rel')}]->({t.get('t')})" for t in triples if t]
    context = (
        "[Chunks]\n" + "\n\n".join(chunks) +
        "\n\n[Entities]\n" + ", ".join(ents) +
        "\n\n[Triples]\n" + "\n".join(triple_lines)
    )
    return context


def qa_graph(query: str, top_k: int = 5, temperature: float | None = None):
    """Graph expansion QA with metrics and adjustable temperature.

    Args:
        query: 問題
        top_k: 檢索擴展的 chunk 數量
        temperature: 若提供，將以該值建立臨時 LLM 以控制隨機性（0 更保守，1 更發散）。
    """
    t0 = time.perf_counter()
    context = expand_context_from_chunks(query, top_k=top_k)
    if not context:
        print("[GraphExpand] 無檢索結果")
        return

    prompt = f"請根據提供的內容回答問題，條列且含依據。\n\n{context}\n\n問題：{query}\n請用繁體中文回答。"

    # 選擇 LLM（支援 per-call 調整 temperature）
    llm_obj = _llm
    if temperature is not None:
        try:
            llm_obj = OllamaLLM(
                model_name=OLLAMA_MODEL,
                model_params={"options": {"temperature": float(temperature)}},
            )
        except Exception as e:
            print(f"[GraphExpand] 建立帶 temperature 的 LLM 失敗，改用預設：{e}")
            llm_obj = _llm

    ans = llm_obj.invoke(prompt).content
    t1 = time.perf_counter()
    latency_ms = (t1 - t0) * 1000.0

    print("\n[GraphExpand] 問題:", query)
    print("[GraphExpand] 回答:\n", ans)
    print(f"Inference latency: {latency_ms:.1f} ms")

    # 以相同 context 構造極簡 probe 估算 tok/s
    probe_prompt = f"請根據以下內容簡短回答：\n\n{context}\n\n問題：{query}\n請用繁體中文回答。"
    tps = _measure_tok_s_hybrid(probe_prompt, model=OLLAMA_MODEL)
    if tps > 0:
        print(f"LLM throughput: {tps:.2f} tok/s (Ollama eval)")
    else:
        print("LLM throughput: N/A (無 eval 統計)")

c:\Users\aint\Desktop\LLM+KB\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name ./models/text2vec-large-chinese. Creating a new one with mean pooling.


⚠️ Fulltext 索引不可用或無法建立，將使用向量-only 檢索做為替代。


In [5]:
# HybridRetriever + Graph expansion QA
from neo4j_graphrag.retrievers.hybrid import HybridRetriever, HybridSearchRanker
from neo4j_graphrag.retrievers import VectorRetriever as _VectorRetriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.llm import OllamaLLM
from neo4j_graphrag.embeddings.sentence_transformers import SentenceTransformerEmbeddings
from neo4j import GraphDatabase
import time
import ollama

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PW = "neo4jgoat"
INDEX_VECTOR = "chunk_embeddings"
INDEX_FTS = "chunk_text_fts"
EMBED_MODEL_DIR = "./models/text2vec-large-chinese"
OLLAMA_MODEL = "llama3:8b-instruct-q4_K_M"

_driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))
_embedder = SentenceTransformerEmbeddings(model=EMBED_MODEL_DIR)

# Ensure the fulltext index exists and is online (idempotent, with procedure fallback)
def _ensure_fulltext_index(driver, index_name: str, label: str, prop: str = "text", strict: bool = False) -> bool:
    """Try to ensure a fulltext index exists. Returns True if exists/created, else False.
    If strict=True, raises when cannot be ensured.
    """
    def _exists(sess) -> bool:
        try:
            rows = sess.run("SHOW INDEXES").data()
            if any(str(r.get("name")) == index_name and (r.get("type") == "FULLTEXT" or r.get("indexType") == "FULLTEXT") for r in rows):
                return True
        except Exception:
            pass
        # Fallback check via procedure
        try:
            r = sess.run("CALL db.index.fulltext.list() YIELD name WHERE name = $name RETURN name", name=index_name).single()
            if r:
                return True
        except Exception:
            pass
        return False

    with driver.session() as session:
        if _exists(session):
            return True
        created = False
        # Try schema DDL (Neo4j 5.x)
        try:
            cypher = f"""
            CREATE FULLTEXT INDEX {index_name} IF NOT EXISTS
            FOR (n:{label}) ON EACH [n.{prop}]
            """
            session.run(cypher)
            created = True
        except Exception:
            created = False
        # Fallback to procedure (Neo4j 4.x/compat)
        if not created:
            try:
                session.run(
                    "CALL db.index.fulltext.createNodeIndex($name, $labels, $props)",
                    name=index_name,
                    labels=[label],
                    props=[prop],
                )
                created = True
            except Exception:
                created = False
        # Wait until online (best-effort)
        try:
            session.run("CALL db.awaitIndexes()")
        except Exception:
            pass
        ok = _exists(session)
        if not ok and strict:
            raise RuntimeError(f"無法建立 Fulltext 索引: {index_name}")
        return ok

_USING_VECTOR_ONLY = False
_has_fts = _ensure_fulltext_index(_driver, INDEX_FTS, label="Chunk", prop="text", strict=False)

# 1) 準備檢索器：若 Fulltext 不可用，則退化成向量-only 檢索
if _has_fts:
    _hybrid = HybridRetriever(
        driver=_driver,
        vector_index_name=INDEX_VECTOR,
        fulltext_index_name=INDEX_FTS,
        embedder=_embedder,
        return_properties=["text", "source", "id"],
    )
else:
    print("⚠️ Fulltext 索引不可用或無法建立，將使用向量-only 檢索做為替代。")
    _USING_VECTOR_ONLY = True
    _hybrid = _VectorRetriever(
        driver=_driver,
        index_name=INDEX_VECTOR,
        embedder=_embedder,
        return_properties=["text", "source", "id"],
    )

_llm = OllamaLLM(model_name=OLLAMA_MODEL)
_rag_hybrid = GraphRAG(retriever=_hybrid, llm=_llm)


def _measure_tok_s_hybrid(prompt: str, model: str = OLLAMA_MODEL) -> float:
    """Estimate tok/s from Ollama eval stats; return -1.0 if unavailable."""
    try:
        r = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}], options={"temperature": 0})
        eval_info = r.get("eval", {}) or r.get("eval_info", {})
        tps = eval_info.get("tokens_per_second") or r.get("tps")
        if isinstance(tps, (int, float)) and tps > 0:
            return float(tps)
        eval_count = r.get("eval_count") or r.get("eval_token_count") or r.get("eval_tokens")
        eval_duration = r.get("eval_duration")  # nanoseconds
        if isinstance(eval_count, (int, float)) and isinstance(eval_duration, (int, float)) and eval_duration:
            seconds = eval_duration / 1e9 if eval_duration > 1e6 else float(eval_duration)
            if seconds > 0:
                return float(eval_count) / seconds
    except Exception:
        pass
    return -1.0


def qa_hybrid(query: str, top_k: int = 5, ranker: str = "naive", alpha: float | None = None, temperature: float | None = None):
    """Hybrid QA with optional temperature to control LLM variability."""
    # choose LLM per call if temperature is provided
    rag_to_use = _rag_hybrid
    if temperature is not None:
        try:
            _llm_tmp = OllamaLLM(
                model_name=OLLAMA_MODEL,
                model_params={"options": {"temperature": float(temperature)}},
            )
            rag_to_use = GraphRAG(retriever=_hybrid, llm=_llm_tmp)
        except Exception as e:
            print(f"[Hybrid] 建立帶 temperature 的 LLM 失敗，改用預設：{e}")
            rag_to_use = _rag_hybrid

    if _USING_VECTOR_ONLY:
        print("[Hybrid] 提示：Fulltext 不可用，現以向量-only 模式執行。")
    cfg = {"top_k": top_k}
    if not _USING_VECTOR_ONLY and ranker == "linear":
        cfg["ranker"] = HybridSearchRanker.LINEAR
        cfg["alpha"] = alpha if alpha is not None else 0.6
    t0 = time.perf_counter()
    resp = rag_to_use.search(query_text=query, retriever_config=cfg, return_context=True)
    t1 = time.perf_counter()
    latency_ms = (t1 - t0) * 1000.0

    print("\n[Hybrid] 問題:", query)
    print("[Hybrid] 回答:", resp.answer)

    # Print latency and estimate tok/s via a small probe
    print(f"Inference latency: {latency_ms:.1f} ms")
    items = getattr(resp, "retriever_result", None)
    # Build a small context snippet for throughput probe
    ctx_texts = []
    if items:
        try:
            # Hybrid items container
            for it in items.items[:min(len(items.items), top_k)]:
                content = getattr(it, "content", None)
                if not content and isinstance(getattr(it, "metadata", None), dict):
                    content = getattr(it, "metadata", {}).get("text")
                if content:
                    ctx_texts.append(str(content))
        except Exception:
            # Vector-only list of dicts
            try:
                for n in items[:min(len(items), top_k)]:
                    ctx_texts.append(str(n.get("text", "")))
            except Exception:
                pass
    ctx_blob = "\n\n".join(ctx_texts)
    probe_prompt = f"請根據以下內容簡短回答：\n\n{ctx_blob}\n\n問題：{query}\n請用繁體中文回答。"
    tps = _measure_tok_s_hybrid(probe_prompt, model=OLLAMA_MODEL)
    if tps > 0:
        print(f"LLM throughput: {tps:.2f} tok/s (Ollama eval)")
    else:
        print("LLM throughput: N/A (無 eval 統計)")

    if items:
        print("[Hybrid] 檢索結果摘要:")
        # 支援 HybridRetriever 的 items 容器或 VectorRetriever 的 list 節點
        try:
            # Hybrid path
            for it in items.items[:min(len(items.items), top_k)]:
                content = getattr(it, "content", None) or it.get("text", "")
                meta = getattr(it, "metadata", None) or it.get("source", "")
                print("-", str(content)[:120].replace("\n", " "), str(meta)[:80])
        except Exception:
            # Vector-only path (list of dicts)
            try:
                for n in items[:min(len(items), top_k)]:
                    print("-", str(n.get("text", ""))[:120].replace("\n", " "), f"(source={n.get('source')})")
            except Exception:
                pass


No sentence-transformers model found with name ./models/text2vec-large-chinese. Creating a new one with mean pooling.


⚠️ Fulltext 索引不可用或無法建立，將使用向量-only 檢索做為替代。


In [6]:
# 診斷與（重新）建立 Fulltext 索引（可選）
from neo4j import GraphDatabase

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PW = "neo4jgoat"
INDEX_FTS = "chunk_text_fts"

_driver_diag = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PW))
with _driver_diag.session() as s:
    print("== SHOW INDEXES ==")
    try:
        for r in s.run("SHOW INDEXES").data():
            print(r)
    except Exception as e:
        print("show indexes error:", e)

    print("\n== Fulltext list ==")
    try:
        for r in s.run("CALL db.index.fulltext.list()").data():
            print(r)
    except Exception as e:
        print("fulltext list error:", e)

    # 嘗試以 DDL 建立（Neo4j 5.x），失敗則以 procedure 建立
    try:
        s.run(f"""
        CREATE FULLTEXT INDEX {INDEX_FTS} IF NOT EXISTS
        FOR (c:Chunk) ON EACH [c.text]
        """)
        print("DDL 建立/確保 Fulltext 索引完成")
    except Exception as e:
        print("DDL 建立失敗，嘗試 procedure：", e)
        try:
            s.run(
                "CALL db.index.fulltext.createNodeIndex($name, $labels, $props)",
                name=INDEX_FTS,
                labels=["Chunk"],
                props=["text"],
            )
            print("procedure 建立 Fulltext 索引完成")
        except Exception as e2:
            print("procedure 亦失敗：", e2)

    # 等待索引 online（最佳努力）
    try:
        s.run("CALL db.awaitIndexes()")
    except Exception:
        pass

    ok = False
    try:
        ok = s.run(
            "CALL db.index.fulltext.list() YIELD name WHERE name = $n RETURN count(*) AS c",
            n=INDEX_FTS,
        ).single().get("c", 0) > 0
    except Exception:
        ok = False

print("Fulltext 可用:" if ok else "Fulltext 仍不可用:", INDEX_FTS)

== SHOW INDEXES ==
{'id': 3, 'name': 'chunk_embeddings', 'state': 'ONLINE', 'populationPercent': 100.0, 'type': 'VECTOR', 'entityType': 'NODE', 'labelsOrTypes': ['Chunk'], 'properties': ['embedding'], 'indexProvider': 'vector-2.0', 'owningConstraint': None, 'lastRead': neo4j.time.DateTime(2025, 9, 22, 11, 32, 17, 818000000, tzinfo=<UTC>), 'readCount': 48}
{'id': 4, 'name': 'chunk_fulltext', 'state': 'ONLINE', 'populationPercent': 100.0, 'type': 'FULLTEXT', 'entityType': 'NODE', 'labelsOrTypes': ['Chunk'], 'properties': ['text'], 'indexProvider': 'fulltext-1.0', 'owningConstraint': None, 'lastRead': neo4j.time.DateTime(2025, 9, 11, 9, 9, 19, 80000000, tzinfo=<UTC>), 'readCount': 10}
{'id': 5, 'name': 'chunk_id_unique', 'state': 'ONLINE', 'populationPercent': 100.0, 'type': 'RANGE', 'entityType': 'NODE', 'labelsOrTypes': ['Chunk'], 'properties': ['id'], 'indexProvider': 'range-1.0', 'owningConstraint': 'chunk_id_unique', 'lastRead': None, 'readCount': 0}
{'id': 7, 'name': 'entity_name_un

In [7]:
RUN_QA_hybrid = True
if RUN_QA_hybrid:
    qa_hybrid("山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答", top_k=5, ranker="linear", alpha=0.6, temperature=0.7)

[Hybrid] 提示：Fulltext 不可用，現以向量-only 模式執行。

[Hybrid] 問題: 山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答
[Hybrid] 回答: 根據提供的文本，山羊缺乏維生素A會導致夜盲症，嚴重甚至瞎眼。
Inference latency: 1560.6 ms

[Hybrid] 問題: 山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答
[Hybrid] 回答: 根據提供的文本，山羊缺乏維生素A會導致夜盲症，嚴重甚至瞎眼。
Inference latency: 1560.6 ms
LLM throughput: 78.18 tok/s (Ollama eval)
[Hybrid] 檢索結果摘要:
- {'id': '0_17', 'text': '生素群,以滿足山羊生理需要。\n\n維生素A:維生素A的缺乏會導致山羊食慾減少,種公羊精液品質降低、缺乏性慾,母羊則發情不易、生殖效率降低,嚴重缺乏亦會造成山羊夜盲之病變。因此1981年版的N {'score': 0.8934812545776367, 'nodeLabels': ['Chunk'], 'id': '4:d0f31978-c03f-40
- {'id': '0_406', 'text': '之總貯存量。\n\n維生素A之功能包括如下：1.維持正常之視力：因維生素A是眼睛視紫質(rhodopsin)色素形成與再生所必需的物質。所以山羊缺乏維生素A會導致夜盲症，嚴重甚至瞎眼。2.織 {'score': 0.8598113059997559, 'nodeLabels': ['Chunk'], 'id': '4:d0f31978-c03f-40
- {'id': '0_434', 'text': '轉換成視覺醇。維生素A在細胞與組織的生長發育上扮演重要的角色。肉羊缺乏維生素A會導致其食慾減少，種公羊會缺乏性慾且精液品質低落。種母羊則會發情不易、生殖效率差，更嚴重的缺乏會造成肉羊眼睛病變 {'score': 0.8474764823913574, 'nodeLabels': ['Chunk'], 'id': '4:d0f31978-c03f-40
- {'id': '0_405', 'text': '痺，尤其後肢僵硬而不能提起。硒由乳汁排出而貯存在皮毛。山羊與其他家

In [3]:
RUN_QA_graph_DEMO = True
if RUN_QA_graph_DEMO:
    qa_graph("山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答", top_k=5,temperature=0.7)


[GraphExpand] 問題: 山羊缺乏維生素A會導致視力出現什麼問題？繁體中文回答
[GraphExpand] 回答:
 根據提供的內容，山羊缺乏維生素A會導致視力出現以下問題：

* 夜盲症：山羊缺乏維生素A會導致夜盲症，山羊在黑暗中無法正常地發現物體。
* 视力下降：維生素A是眼睛視紫質(rhodopsin)色素形成與再生的必要物質，因此山羊缺乏維生素A會導致視力下降。

這兩個問題都是由於山羊的維生素A水平不足所引起的，並且如果缺乏維生素A太久，可能會導致瞎眼。
Inference latency: 4473.8 ms
LLM throughput: 76.81 tok/s (Ollama eval)


## 🔍 利用圖譜關係的進階檢索模式

相較於 QA-only 的純向量檢索，以下兩種模式能充分利用已建立的知識圖譜關係：

### 1️⃣ Graph Expansion 模式 (推薦)
**最佳選擇**：直接利用實體關係和三元組進行推理

**優勢**：
- 🧠 **智能推理**：通過 `MENTIONS` 關係找到相關實體
- 🔗 **關係擴展**：利用 `RELATION` 三元組豐富上下文
- 📊 **結構化資訊**：提供 [Chunks] + [Entities] + [Triples] 完整脈絡
- 🎯 **精準回答**：基於知識圖譜的邏輯推理

**使用方式**：
```python
# 基礎用法
qa_graph("你的問題", top_k=5)

# 控制創意度（temperature）
qa_graph("你的問題", top_k=5, temperature=0.0)  # 保守回答
qa_graph("你的問題", top_k=5, temperature=0.8)  # 創意回答
```

### 2️⃣ Hybrid 模式
**平衡選擇**：結合向量檢索和全文檢索

**優勢**：
- 🔍 **雙重檢索**：向量語意 + 關鍵字匹配
- ⚡ **高效能**：比純圖遍歷更快
- 🎛️ **可調權重**：控制向量vs全文的比重

**使用方式**：
```python
# 基礎混合檢索
qa_hybrid("你的問題", top_k=5)

# 線性加權檢索（推薦）
qa_hybrid("你的問題", top_k=5, ranker="linear", alpha=0.6)

# 控制創意度
qa_hybrid("你的問題", top_k=5, temperature=0.7)
```

### 🆚 三種模式比較

| 特性 | QA-only | Hybrid | Graph Expansion |
|------|---------|--------|-----------------|
| **檢索方式** | 純向量 | 向量+全文 | 向量+圖譜關係 |
| **使用圖譜** | ❌ | ❌ | ✅ |
| **推理能力** | 基礎 | 中等 | 高級 |
| **回答品質** | 良好 | 很好 | 優秀 |
| **執行速度** | 最快 | 快 | 中等 |
| **適用場景** | 簡單問答 | 關鍵字+語意 | 複雜推理 |

### 💡 建議使用策略

1. **複雜推理問題** → 使用 `qa_graph()`
2. **需要精確匹配** → 使用 `qa_hybrid()` 
3. **快速簡單查詢** → 使用 `qa_only()`

In [5]:
# 🧪 進階檢索模式示範與比較
print("=== 🔍 進階檢索模式比較示範 ===\n")

# 測試問題
test_query = "山羊缺乏維生素A會導致視力出現什麼問題？"

print("🚀 1. Graph Expansion 模式 (利用圖譜關係)")
print("=" * 50)
qa_graph(test_query, top_k=5, temperature=0.3)

print("\n" + "=" * 80 + "\n")

print("🔍 2. Hybrid 模式 (向量 + 全文檢索)")
print("=" * 50)
qa_hybrid(test_query, top_k=5, ranker="linear", alpha=0.6, temperature=0.3)

print("\n" + "=" * 80 + "\n")
print("💡 比較結果：")
print("- Graph Expansion 提供更豐富的實體關係和三元組資訊")
print("- Hybrid 模式結合語意和關鍵字匹配，檢索更全面")
print("- 兩種模式都比 QA-only 提供更深入的上下文理解")

=== 🔍 進階檢索模式比較示範 ===

🚀 1. Graph Expansion 模式 (利用圖譜關係)

[GraphExpand] 問題: 山羊缺乏維生素A會導致視力出現什麼問題？
[GraphExpand] 回答:
 根據提供的內容，山羊缺乏維生素A會導致夜盲症（夜盲症）。維生素A是眼睛視紫質(rhodopsin)色素形成與再生的必要物質，因此缺乏維生素A會導致山羊失去視力。
Inference latency: 6731.4 ms

[GraphExpand] 問題: 山羊缺乏維生素A會導致視力出現什麼問題？
[GraphExpand] 回答:
 根據提供的內容，山羊缺乏維生素A會導致夜盲症（夜盲症）。維生素A是眼睛視紫質(rhodopsin)色素形成與再生的必要物質，因此缺乏維生素A會導致山羊失去視力。
Inference latency: 6731.4 ms
LLM throughput: 79.82 tok/s (Ollama eval)


🔍 2. Hybrid 模式 (向量 + 全文檢索)
LLM throughput: 79.82 tok/s (Ollama eval)


🔍 2. Hybrid 模式 (向量 + 全文檢索)
[Hybrid] 提示：Fulltext 不可用，現以向量-only 模式執行。
[Hybrid] 提示：Fulltext 不可用，現以向量-only 模式執行。

[Hybrid] 問題: 山羊缺乏維生素A會導致視力出現什麼問題？
[Hybrid] 回答: According to the provided context, mountain goats lacking vitamin A will experience night blindness, which can lead to severe consequences such as complete loss of vision.
Inference latency: 3100.1 ms

[Hybrid] 問題: 山羊缺乏維生素A會導致視力出現什麼問題？
[Hybrid] 回答: According to the provided context, mountain goats lacking vitamin A will experience ni

In [ ]:
# 🎛️ 參數調優示範
print("=== 🎛️ 不同參數設定的效果比較 ===\n")

test_query = "山羊缺乏維生素A會導致什麼健康問題？"

print("🧊 低溫度 (temperature=0.0) - 保守回答")
print("-" * 40)
qa_graph(test_query, top_k=3, temperature=0.0)

print("\n" + "=" * 60 + "\n")

print("🔥 高溫度 (temperature=0.9) - 創意回答")
print("-" * 40)
qa_graph(test_query, top_k=3, temperature=0.9)

print("\n" + "=" * 60 + "\n")

print("⚖️ Hybrid 模式權重調整示範")
print("-" * 40)
print("📊 Alpha=0.3 (偏重全文檢索)")
qa_hybrid(test_query, top_k=3, ranker="linear", alpha=0.3, temperature=0.5)

print("\n" + "-" * 40 + "\n")
print("📊 Alpha=0.8 (偏重向量檢索)")
qa_hybrid(test_query, top_k=3, ranker="linear", alpha=0.8, temperature=0.5)